In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/comp4211/deepfillv2-pytorch-master.zip -d /content/drive/MyDrive/comp4211/

In [ ]:
!unzip /content/drive/MyDrive/comp4211/input.zip -d /content/drive/MyDrive/comp4211/

In [ ]:
test_path="/content/drive/MyDrive/comp4211/deepfillv2-pytorch-master/test.py"
img_path="/content/drive/MyDrive/comp4211/deepfillv2-pytorch-master/examples/inpaint/1792_27904.png"
mask_path="/content/drive/MyDrive/comp4211/deepfillv2-pytorch-master/examples/inpaint/1792_27904(2).png"
out_path="/content/drive/MyDrive/comp4211/deepfillv2-pytorch-master/out_file/case1_out_test.png"
check_path="/content/drive/MyDrive/comp4211/deepfillv2-pytorch-master/pretrained/states_tf_places2.pth"


In [ ]:
import numpy as np
import cv2
from queue import Queue

def process_mask(mask):
    height, width = mask.shape
    visited = np.zeros_like(mask, dtype=bool)
    result_mask = np.zeros_like(mask)
    Q = Queue()
    for x in range(height):
        for y in range(width):
            if (x == 0 or y == 0 or x == height - 1 or y == width - 1) :
              if(mask[x][y] == 255):
                Q.put((x, y))
                visited[x][y] = True


    dirs = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    while not Q.empty():
        x, y = Q.get()
        result_mask[x][y] = 1
        for dx, dy in dirs:
            nx, ny = x + dx, y + dy
            if nx < 0 or nx >= height or ny < 0 or ny >= width:
                continue
            if mask[nx][ny] == 255 and not visited[nx][ny]:
                Q.put((nx, ny))
                visited[nx][ny] = True
    return result_mask

In [ ]:
# enviroment generation
import os
import subprocess
import PIL.Image as Image
import cv2
import numpy as np
import torch
from torchvision import transforms
import torch.nn.functional as F
input_path="/content/drive/MyDrive/comp4211/input"
mask_path="/content/drive/MyDrive/comp4211/prediction"
out_path="/content/drive/MyDrive/comp4211/deep_fill_output_augment"

i=0
for name in os.listdir(mask_path):
  input_image=os.path.join(input_path,name)
  input_mask=os.path.join(mask_path,name)
  img=cv2.imread(input_mask, 0)
  img=torch.Tensor(img)
  img=process_mask(img)
  size=img.shape
  img=torch.Tensor(img)
  padding = (50, 50, 50, 50)
  img_padded = F.pad(img, padding, "constant", 1)
  img_padded=img_padded.numpy()
  numpy_array = (img_padded*255).astype(np.uint8)
  pil_image = Image.fromarray(numpy_array,'L')
  pil_image=pil_image.resize(size)
  pil_image.save('aug_mask.png')
  input_mask='aug_mask.png'
  in_img=cv2.imread(input_image, cv2.IMREAD_COLOR)
  padding = (50, 50, 50, 50)
  in_img=torch.Tensor(in_img)
  in_img=torch.permute(in_img,(2,0,1))
  in_img = F.pad(in_img, padding, "constant", 0)
  in_img=torch.permute(in_img,(1,2,0))
  in_img=in_img.numpy()
  resized_input=cv2.resize(in_img,size)
  resized_input=resized_input[:, :, :3]
  img_bgr = cv2.cvtColor(resized_input, cv2.COLOR_RGB2BGR)
  cv2.imwrite("input_image.png",img_bgr)
  input_image="input_image.png"
  out_name=os.path.join(out_path,name)
  params={"--mask":input_mask,"--checkpoint":check_path,"--image":input_image,"--out":out_name}
  params_list = []
  for key, value in params.items():
    params_list.append(key)
    params_list.append(value)

# 把主命令和参数放在一个列表中
  command = ['python3', '/content/drive/MyDrive/comp4211/deepfillv2-pytorch-master/test.py'] + params_list

# 在subprocess.run中运行
  subprocess.run(command)
  print(out_name)
  i+=1



/content/drive/MyDrive/comp4211/deep_fill_output_augment/8192_17664.png
/content/drive/MyDrive/comp4211/deep_fill_output_augment/8192_31232.png
/content/drive/MyDrive/comp4211/deep_fill_output_augment/8192_22528.png
/content/drive/MyDrive/comp4211/deep_fill_output_augment/8192_30976.png
/content/drive/MyDrive/comp4211/deep_fill_output_augment/8192_23552.png
/content/drive/MyDrive/comp4211/deep_fill_output_augment/7936_21504.png
/content/drive/MyDrive/comp4211/deep_fill_output_augment/7936_30464.png
/content/drive/MyDrive/comp4211/deep_fill_output_augment/8192_26368.png
/content/drive/MyDrive/comp4211/deep_fill_output_augment/7936_19968.png
/content/drive/MyDrive/comp4211/deep_fill_output_augment/7936_27136.png
/content/drive/MyDrive/comp4211/deep_fill_output_augment/8192_18688.png
/content/drive/MyDrive/comp4211/deep_fill_output_augment/7936_20992.png
/content/drive/MyDrive/comp4211/deep_fill_output_augment/7936_29440.png
/content/drive/MyDrive/comp4211/deep_fill_output_augment/8192_24

KeyboardInterrupt: 

In [ ]:
# building erase
import os
import subprocess
import PIL.Image as Image
import cv2
import numpy as np
#import torch
input_path="/content/drive/MyDrive/comp4211/input"
mask_path="/content/drive/MyDrive/comp4211/prediction"
out_path="/content/drive/MyDrive/comp4211/deep_fill_output_recon"
if not os.path.exists(out_path):
  os.mkdir(out_path)
i=0
for name in os.listdir(mask_path):
  input_image=os.path.join(input_path,name)
  input_mask=os.path.join(mask_path,name)
  img=cv2.imread(input_mask, 0)
  img=img.astype(np.uint8)
  kernel = np.ones((3,3),np.uint8)
  dilation = cv2.dilate(img,kernel,iterations = 15)
  cv2.imwrite('dilation.png', dilation)
  input_mask='dilation.png'
  out_name=os.path.join(out_path,name)
  params={"--mask":input_mask,"--checkpoint":check_path,"--image":input_image,"--out":out_name}
  params_list = []
  for key, value in params.items():
    params_list.append(key)
    params_list.append(value)
  command = ['python3', '/content/drive/MyDrive/comp4211/deepfillv2-pytorch-master/test.py'] + params_list
  subprocess.run(command)
  print(i)
  i+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226


KeyboardInterrupt: 